In [1]:
from qutip import *

import numpy as np

import matplotlib.pyplot as plt

from numba import jit, complex128

(CVXPY) Jan 25 02:03:33 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


In [2]:
def Hxyz(N, Jx, Jy, Jz, b, periodic=False):
    """Generates an XYZ spin chain Hamiltonian with N spins"""
    J_list = [Jx, Jy, Jz]
    sigma_list = [sigmax(), sigmay(), sigmaz()]
    if N <= 1:
        return qeye(2**N)
    Ham = 0
    for i in range(N-1):
        for j in range(3):
            spin_list = [sigma_list[j] if k == i or k == i+1 else qeye(2) for k in range(N)]
            Ham += J_list[j]*tensor(spin_list)
    for i in range(N):
        spin_list = [sigmaz() if k == i else qeye(2) for k in range(N)]
        Ham += b*tensor(spin_list)
    if periodic:
        for j in range(3):
            spin_list = [sigma_list[j] if k == 0 or k == N-1 else qeye(2) for k in range(N)]
            Ham += J_list[j]*tensor(spin_list)
    return Ham

In [3]:
@jit(nopython=True)
def manual_trace(matrix):
    trace = 0.0 + 0.0j  # Explicitly specifying as complex
    for i in range(matrix.shape[0]):
        trace += matrix[i, i]
    return trace

@jit(nopython=True)
def partial_trace_final(state, final_subsys_size, remainder_size):
    if state.ndim != 2 or state.shape[0] != final_subsys_size * remainder_size:
        raise ValueError("Invalid state dimensions")

    reshaped_state = state.reshape((remainder_size, final_subsys_size, remainder_size, final_subsys_size))
    
    # Manual trace computation with consistent data types
    traced_state = np.zeros((remainder_size, remainder_size), dtype=np.complex128)
    for i in range(remainder_size):
        for j in range(remainder_size):
            traced_state[i, j] = manual_trace(reshaped_state[i, :, j, :])

    normalization = manual_trace(traced_state)
    if normalization != 0:
        traced_state /= normalization

    return traced_state

In [4]:
# Sample 4x4 density matrix for a 2-qubit system
state = np.array([[0.5, 0.0, 0.0, 0.5],
                  [0.0, 0.0, 0.0, 0.0],
                  [0.0, 0.0, 0.0, 0.0],
                  [0.5, 0.0, 0.0, 0.5]])

# Subsystems to trace out (second qubit)
subsystems = [1]

# Dimension list for each subsystem (2-qubit system)
dim_list = [2, 2]

# Now you can call the partial_trace function with these inputs
state2 = np.array([1/2, -1j/2, 1j/2, 1/2])
result1 = partial_trace_final(state, 2, 2)
print(result1)

[[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]]


In [ ]:
#set all the relevant parameters
bfield = 0.5
J = 1.
gamma = 1.
Delta = 0.

num_collision = 1200
collision_duration = 0.5
timestep = 0.01

num_spins = 4

In [ ]:
#generate initial states
np.random.seed(10) #10 best for 4 spins
psi_0 = tensor([rand_ket_haar(2) for _ in range(num_spins)])
rho_S_0 = psi_0 * psi_0.dag()

beta = 3.5 #temperature
g_prob = 0.5*(1+np.tanh(beta))
rho_A = g_prob * (basis(2,1) * basis(2,1).dag()) + (1-g_prob) * (basis(2,0) * basis(2,0).dag())

#expectation values to calculate
expec_list_S = [tensor([sigmax() if k == i else qeye(2) for k in range(num_spins)]) 
              for i in range(num_spins)] + [tensor([sigmay() if k == i else qeye(2) for k in range(num_spins)]) 
              for i in range(num_spins)]

#system and interaction Hamiltonians
H_S = -Hxyz(num_spins, J, J*gamma, J*Delta, bfield, periodic=True)
H_SA = 0.5*(tensor(sigmax(), *[qeye(2) for _ in range(num_spins - 1)], sigmax()) 
          + tensor(sigmay(), *[qeye(2) for _ in range(num_spins - 1)], sigmay()))